In [19]:
from utils import load_config
from llm import *
from prompt import *
from harness import *

In [18]:
from sys import modules
del modules['harness']
del modules['llm']
del modules['prompt']
del modules['utils']

In [25]:
CONFIG = load_config("/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/lib_mul.yaml")
CONFIG

{'function_name': 'lib_mul',
 'is_c_code': True,
 'lib_files': ['/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/src/library.c',
  '/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/src/library.h'],
 'harness': '/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/harnesslib_mul.c',
 'harness_bin': '/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/harnesslib_mul'}

In [81]:
# function_name="ares_create_query"
# IS_C_CODE= True
# c_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.c')
# h_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.h')
# lib_files = c_files
# HARNESS = 'harness.c'
# HARNESS_BIN = 'harness'

In [3]:
import subprocess

def has_coverage():
    seconds_to_wait = 15
    print(f'checking that there is some coverage (Running the fuzzer for {seconds_to_wait}) seconds')
    # print(datetime.now())
    # r = subprocess.Popen(['afl-fuzz', '-i', 'seed', '-o', 'out', '--', f'./{HARNESS_BIN}'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    # time.sleep(seconds_to_wait)
    # r.terminate()
    # try:
    #     r.wait(timeout=2)  # Give some time to clean up resources
    # except subprocess.TimeoutExpired:
    #     r.kill()  # Force kill if it's still not terminating
    # print(datetime.now())
    !timeout 5s afl-fuzz -i seed -o out -- ./harness > /dev/null
    result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stats = extract_summary_stats(result.stdout)
    print(f"{stats['coverage reached'][:-1]}")
    if float(stats['coverage reached'][:-1]) == 0:
        print('Trying to fuzz for 60 seconds')
        !timeout 60s afl-fuzz -i seed -o out -- ./harness > /dev/null
        result = subprocess.run(['afl-whatsup', 'out/'], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stats = extract_summary_stats(result.stdout)
        print(f"{stats['coverage reached'][:-1]}")
    return not float(stats['coverage reached'][:-1]) == 0

In [94]:
# first_answer = get_answer(initial_prompt)
len(initial_prompt)

1685

In [ ]:
# get_answer(initial_prompt)

In [29]:
# GENERATE HARNESS

tentatives = 10
initial_prompt = generate_initial_prompt(CONFIG)
first_answer = get_answer(initial_prompt)
extract_and_save_harness(first_answer, CONFIG['harness'])
prev_prompt = initial_prompt
prev_res = first_answer[0]['generated_text']

compile_command = ['afl-gcc-fast', '-fsanitize=fuzzer', 
                   CONFIG['harness'], '/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/src/library.c', '-o', CONFIG['harness_bin']]

while tentatives:
    print(f"{tentatives=}")
    tentatives = tentatives -1
    # checking that harness is compilable
    compile_res = compile(compile_command)
    if compile_res[0]:
    # checking that does not crash on seed
        run_res = run_seed()
        if run_res[0]:
            # checking that there is some coverage
            if has_coverage():
                break
            else:
                prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "When running afl-fuzz over the generated code, the Coverage reached value is 0%. It probably means that the input from the fuzzer is not correclty passed to the fuzzed function. Can you regenerate the code to fiX this issue?\n")
                prev_res = get_answer(prev_prompt)
                extract_and_save_harness(prev_res, CONFIG['harness'])
        else:
            print(f"{run_res[1]=}")
            prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "The generated code when run with input AAAA is crashing with this error\n" + run_res[1] + '\ncan you regenerate the previous code to fix the issue?')
            prev_res = get_answer(prev_prompt)
            extract_and_save_harness(prev_res, CONFIG['harness'])
    else:
        print(f"{compile_res[1]=}")
        prev_prompt = followup_prompt(prev_prompt, prev_res[0]['generated_text'], "can you regenerate the previous code to fix the following compilation error:\n" + compile_res[1])
        prev_res = get_answer(prev_prompt)
        extract_and_save_harness(prev_res, CONFIG['harness'])
        # break

  ```c
    #include <stdint.h>
    #include <stdlib.h>
    #include "library.h"
    
    int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {
    	lib_mul(Data[0], Data[1]);
    	return 0;
    }
    ```
tentatives=10
checking that harness is compilable
compile_command=['afl-gcc-fast', '-fsanitize=fuzzer', '/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/harnesslib_mul.c', '/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/src/library.c', '-o', '/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/harnesslib_mul']
Compilation failed.
Errors: /home/sagemaker-user/harnessgeneration/evaluation/simple_lib/harnesslib_mul.c:3:14: fatal error: library.h: No such file or directory
    3 |     #include "library.h"
      |              ^~~~~~~~~~~
compilation terminated.

compile_res[1]='/home/sagemaker-user/harnessgeneration/evaluation/simple_lib/harnesslib_mul.c:3:14: fatal error: library.h: No such file or directory\n    3 |     #include "library.h"\n 

TypeError: string indices must be integers

In [30]:
prev_res

'  ```c\n    #include <stdint.h>\n    #include <stdlib.h>\n    #include "library.h"\n    \n    int LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {\n    \tlib_mul(Data[0], Data[1]);\n    \treturn 0;\n    }\n    ```'

In [28]:
prev_res

[{'generated_text': '  Certainly! Here is the corrected code:\n```c\n#include <stdint.h>\n#include <stdlib.h>\n#include "simple_lib/library.h"\n\nint LLVMFuzzerTestOneInput(const uint8_t *Data, size_t Size) {\n    char *data = (char *)Data;\n    ssize_t len = (ssize_t)Size;\n    lib_echo(data, len);\n    return 0;\n}\n```\nThe error was caused by the missing `#include "simple_lib/library.h"` directive, which was needed to include the header file for the `lib_echo` function.'}]

In [56]:
# r = get_answer(fix_compilation_prompt)

In [71]:
c_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.c')
h_files = find_files("/home/sagemaker-user/fuzzer-test-suite/c-ares-CVE-2016-5180/SRC", '.h')